In [27]:
import numpy as np
import pandas as pd

In [28]:
df = pd.read_csv("Train_3.csv")
df.drop(columns = ["employee_id","Unnamed: 0"],inplace = True)

In [19]:
import lightgbm as ltb

from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error
#from sklearn.metrics import f1_score 

#X_train,X_test,y_train,y_test = tts(df.drop('is_promoted', axis = 1),df['is_promoted'],test_size = 0.3,random_state = 1)

from sklearn.preprocessing import PolynomialFeatures
PF = PolynomialFeatures(2);
data_p = PF.fit_transform(df.drop('is_promoted',axis = 1))
Xp_train,Xp_test,y_train,y_test = tts(data_p,df['is_promoted'],test_size = 0.3,random_state = 1)

#Feature Scaling
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#Xp_train = sc.fit_transform(Xp_train)
#Xp_test = sc.transform(Xp_test)

model = ltb.LGBMClassifier()
model.fit(Xp_train,y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [20]:
from sklearn.metrics import f1_score 

pred1 = model.predict(Xp_train)
pred = model.predict(Xp_test)
print(f1_score(pred1,y_train))
print(f1_score(pred,y_test))

0.5566739606126915
0.525554981930821


In [21]:
dfr = pd.read_csv("Test_2.csv")
df1 = dfr["employee_id"]
#dfr.drop(columns = ["employee_id"],inplace = True)
dfr.drop(columns = ["employee_id","Unnamed: 0"],inplace = True)
#dfr = dfr.drop(columns = ["other", "referred" , "sourcing"])
data_p1 = PF.fit_transform(dfr)
t=model.predict(data_p1)

In [22]:
T = pd.DataFrame(t)
dfr1 = pd.concat((df1,T),axis = 1)

In [23]:
dfr1.to_csv("lgbm_fsc_poly3_1.csv")

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------#

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------#

In [ ]:
#-------------------------------------------------------------------------------------------------------------------------#

In [85]:
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import mean_squared_error
#from sklearn.metrics import f1_score 

#X_train,X_test,y_train,y_test = tts(df.drop('is_promoted', axis = 1),df['is_promoted'],test_size = 0.3,random_state = 1)

from sklearn.preprocessing import PolynomialFeatures
PF = PolynomialFeatures(2);
data_p = PF.fit_transform(df.drop('is_promoted',axis = 1))
Xp_train,Xp_test,y_train,y_test = tts(data_p,df['is_promoted'],test_size = 0.3,random_state = 1)


#Feature Scaling
#from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
#Xp_train = sc.fit_transform(Xp_train)
#Xp_test = sc.transform(Xp_test)

In [86]:
from sklearn.metrics import f1_score

def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.round(y_hat) # scikits f1 doesn't like probabilities
    return 'f1', f1_score(y_true, y_hat), True

In [87]:
import lightgbm as lgb
def run_lgb(X_train, X_test, y_train, y_test, test_df):
    d_train = lgb.Dataset(Xp_train, label=y_train)
    params = {
       "objective" : "binary",
       "n_estimators":10000,
       "reg_alpha" : 0.1,
       "reg_lambda":0.1,
       "n_jobs":-1,
       "colsample_bytree":.8,
       "min_child_weight":8,
       "subsample":0.8715623,
       "min_data_in_leaf":100,
       "nthread":4,
       "metric" : "f1",
       "num_leaves" : 600,
       "learning_rate" : 0.01,
       "verbosity" : -1,
       "seed": 120,
       "max_bin":60,
       'max_depth':15,
       'min_gain_to_split':.0222415,
       'scale_pos_weight':2
        }
#clf = lgb.train(params, d_train, 100)

    lgtrain = lgb.Dataset(Xp_train, label=y_train)
    lgval = lgb.Dataset(Xp_test, label=y_test)
    evals_result = {}
    clf = lgb.train(params, lgtrain, 10000, 
                      valid_sets=[lgtrain, lgval], 
                      early_stopping_rounds=100, 
                      verbose_eval=100, 
                      evals_result=evals_result,feval=lgb_f1_score)
    pred_test_y = clf.predict(test_df, num_iteration=clf.best_iteration)
    return pred_test_y, clf, evals_result

In [89]:
pred_test, clf, evals_result = run_lgb(Xp_train, Xp_test, y_train, y_test, data_p1)
print("LightGBM Training Completed...")

C:\Users\Shashank\Anaconda3\lib\site-packages\lightgbm\engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\Shashank\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Training until validation scores don't improve for 100 rounds
[100]	training's f1: 0.440718	valid_1's f1: 0.446797
[200]	training's f1: 0.515044	valid_1's f1: 0.508598
[300]	training's f1: 0.545377	valid_1's f1: 0.51659
[400]	training's f1: 0.562053	valid_1's f1: 0.530735
[500]	training's f1: 0.569583	valid_1's f1: 0.530348
[600]	training's f1: 0.578283	valid_1's f1: 0.531558
[700]	training's f1: 0.590788	valid_1's f1: 0.536992
[800]	training's f1: 0.600163	valid_1's f1: 0.533333
Early stopping, best iteration is:
[701]	training's f1: 0.591532	valid_1's f1: 0.537255
LightGBM Training Completed...


In [88]:
dfr = pd.read_csv("Test_2.csv")
df1 = dfr["employee_id"]
#dfr.drop(columns = ["employee_id"],inplace = True)
dfr.drop(columns = ["employee_id","Unnamed: 0"],inplace = True)
#dfr = dfr.drop(columns = ["other", "referred" , "sourcing"])
data_p1 = PF.fit_transform(dfr)

In [90]:
pred_test.size

23490

In [91]:
for i in range(0,23490):
    if pred_test[i]>=.44:       # setting threshold to .5
       pred_test[i]=1
    else:  
       pred_test[i]=0
pred_test

array([1., 1., 1., ..., 1., 1., 1.])

In [92]:
T = pd.DataFrame(pred_test)
dfr1 = pd.concat((df1,T),axis = 1)

In [93]:
dfr1.to_csv("lgbm_paramtune_featurescl_0.4thresh_poly2_1.csv")